In [75]:
import time
import urllib
import os.path
import sys
import os
import numpy as np
import matplotlib.pylab as plt
from os import listdir
import numpy as np
from PIL import Image
from sklearn.ensemble import RandomForestClassifier

In [76]:
path = os.path.join(os.getcwd(), "archive\Training")
images = []
classifiers = []

In [77]:
gliomaPath = os.path.join(path, 'glioma_tumor')
files = os.listdir(gliomaPath)
for file in files:
    try:
        with Image.open(gliomaPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(1)
    except FileNotFoundException:
        print(file)
        break

In [78]:
meningPath = os.path.join(path, 'meningioma_tumor')
files = os.listdir(meningPath)
for file in files:
    try:
        with Image.open(meningPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(2)
    except FileNotFoundException:
        print(file)
        break

In [79]:
noTumorPath = os.path.join(path, 'glioma_tumor')
files = os.listdir(noTumorPath)
for file in files:
    try:
        with Image.open(noTumorPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(0)
    except FileNotFoundException:
        print(file)
        break

In [80]:
pituitaryPath = os.path.join(path, 'pituitary_tumor')
files = os.listdir(pituitaryPath)
for file in files:
    try:
        with Image.open(pituitaryPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(3)
    except FileNotFoundException:
        print(file)
        break

In [86]:
rf = RandomForestClassifier(max_depth = 20, random_state = 912983)

In [87]:
rf.fit(images, classifiers)

RandomForestClassifier(max_depth=20, random_state=912983)

In [73]:
path = os.path.join(os.getcwd(), "archive\Testing")
testImages = []
testClassifiers = []

gliomaPath = os.path.join(path, 'glioma_tumor')
files = os.listdir(gliomaPath)
for file in files:
    try:
        with Image.open(gliomaPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            testImages.append(np.asarray(image).flatten())
            testClassifiers.append(1)
    except FileNotFoundException:
        print(file)
        break
        
meningPath = os.path.join(path, 'meningioma_tumor')
files = os.listdir(meningPath)
for file in files:
    try:
        with Image.open(meningPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            testImages.append(np.asarray(image).flatten())
            testClassifiers.append(2)
    except FileNotFoundException:
        print(file)
        break
        
noTumorPath = os.path.join(path, 'glioma_tumor')
files = os.listdir(noTumorPath)
for file in files:
    try:
        with Image.open(noTumorPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            testImages.append(np.asarray(image).flatten())
            testClassifiers.append(0)
    except FileNotFoundException:
        print(file)
        break
        
pituitaryPath = os.path.join(path, 'pituitary_tumor')
files = os.listdir(pituitaryPath)
for file in files:
    try:
        with Image.open(pituitaryPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            testImages.append(np.asarray(image).flatten())
            testClassifiers.append(3)
    except FileNotFoundException:
        print(file)
        break

In [88]:
rf.score(testImages, testClassifiers)

0.4498714652956298

In [85]:
rf.predict(testImages)

array([2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 2, 0, 0, 1, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 2, 0, 0, 1,
       1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,